<a href="https://colab.research.google.com/github/michalis0/DataMining_and_MachineLearning/blob/master/week3/Advanced_Pandas_Operations_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining and Machine Learning - Week 3
# Advanced Pandas Operations - Aggregation

### Table of Contents
###### 1. Simple Aggregation
###### 2. Advanced aggregation topics and chaining
###### 3. Selecting using an aggregating dimension

Note: This notebook follows the sildes of the pdf file <a href='https://github.com/michalis0/DataMining_and_MachineLearning/blob/master/week3/Walk-Through-Tutorial/3.%20Pandas_Aggregate.pdf'>3. Pandas_Aggregate.pdf</a> provided in the github repository.

In [91]:
# Import required packages
import pandas as pd

In [92]:
# Read in data
myData = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/week3/data/transactions_students.csv", sep=",")
myData.head(3)

,Customer,TransDate,Quantity,PurchAmount,Cost,TransID
0,149332,15.11.2005,1,199.95,107.0,127998739
1,172951,29.08.2008,1,199.95,108.0,128888288
2,120621,19.10.2007,1,99.95,49.0,125375247


In [93]:
# Get basic info on the dataset
myData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223191 entries, 0 to 223190
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Customer     223191 non-null  int64  
 1   TransDate    223191 non-null  object 
 2   Quantity     223191 non-null  int64  
 3   PurchAmount  223191 non-null  float64
 4   Cost         223191 non-null  float64
 5   TransID      223191 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 10.2+ MB


In [ ]:
# Adjust the format of column "TransDate" to datetime.
myData["TransDate"]  = pd.to_datetime(myData["TransDate"], dayfirst=True)

# Alternative conversion: Convert date from string to date times
# import dateutil
# myData["TransDate"] = myData["TransDate"].apply(dateutil.parser.parse, dayfirst=True)

myData.info()

In [ ]:
myData["TransDate"].head()

In [ ]:
pd.options.display.max_rows = 10 # from this row on, show 10 rows for Pandas Data Frames

### 1. Simple Aggregations

#### 1.0 Apply an aggregating function to an entire series
###### Sum all `PurchAmount` across all observations.

In [ ]:
total = myData["PurchAmount"].sum()
print(f"{total:,.3f}")

#### 1.1 Apply an aggregating function to a variable by an aggregating dimension 

Typical problem: the dataset includes several rows (transactions) for each customer. But we would like to sum up certain variable(s) by customer (one row per customer).

###### Sum up the `PurchAmount` by `Customer` for every different customer

In [ ]:
myData.groupby("Customer", as_index=False)["PurchAmount"].sum()

In [ ]:
# alternative option: using agg and then renaming the aggregation column to AggPurch
myData.groupby("Customer")["PurchAmount"].agg({"sum"}).rename(columns={"sum":"AggPurch"}).reset_index()

#### 1.2 Apply multiple aggregation functions to a variable by an aggregating dimension
###### Sum and select max of `PurchAmount` by `Customer`

In [ ]:
myData.groupby("Customer", as_index=False)["PurchAmount"].agg({"AggPurch": "sum", "Purch_max": "max"})

#### 1.3 Create new columns in your existing data frame with the `transform()`-function
Here we add a new column `AggPurch` which holds for every customer the total of her purchases.

In [ ]:
myData["AggPurch"] = myData.groupby("Customer")["PurchAmount"].transform(sum)
myData.head()

#### 1.4 Apply a/ multiple aggregating function(s) to multiple variables by an aggregating dimension
###### Apply various aggregation functions to `PurchAmount` and `Quantity` grouped by `Customer`

In [ ]:
myData.groupby("Customer", as_index=False).agg({"PurchAmount":["max","sum"], "Quantity":"sum"})

#### 1.5 Apply an aggregating function to multiple variables by multiple aggregating dimensions
###### Sum `PurchAmount` and sum `Quantity` aggregated by `Customer` and `TransDate`

In [ ]:
myData.groupby(["Customer","TransDate"])[["PurchAmount","Quantity"]].sum()

#### 1.6 Apply an aggregating function to a variable by an aggregating dimension to a selection of rows
###### Select rows 2 to 5 and sum `PurchAmount` by `Customer`

In [ ]:
myData.iloc[1:6].groupby("Customer", as_index=False)["PurchAmount"].sum()

#### 1.7 Exercises
###### 1. Sum `PurchAmount` by `Customer` and `TransDate` (hint: group by both `Customer` and `TransDate`).
###### 2. Count the number of transactions by `Customer` (hint: use the `count()` function)

In [ ]:
# 1. [your code here]
myData.groupby(["Customer", "TransDate"])["PurchAmount"].sum().reset_index()

In [ ]:
# 2. [your code here]
myData.groupby("Customer", as_index=False)["TransID"].count()

### 2. Advanced aggregation topics and chaining
#### 2.1 Aggregate a variable by a transformed aggregating dimension
###### Sum `PurchAmount` by month of each year

In [ ]:
myData.groupby(myData["TransDate"].dt.to_period("M"))[["PurchAmount"]].sum()

#### Sidenote: Chaining saves memory and is faster
Sum `PurchAmount` by `Customer` and select Customers with aggregated sums greater than 100.

In [ ]:
myData.groupby("Customer")[["PurchAmount"]].sum()[lambda x: x >= 100].dropna(axis=0)

In [ ]:
# alternative
myData2 = myData.groupby("Customer", as_index=False)["PurchAmount"].agg({"AggPurch": "sum"})
myData2.loc[myData2["AggPurch"]>=100]

#### 2.2 Exercises
###### 1. Aggregate the purchase amount (sum) of all transactions per customer on a yearly basis for year 2007 and 2008.
###### 2. How many customers purchased more than 50$ in total between 2008 and 2009?

In [ ]:
# Question 1.
# Part1 use the loc function and select the year 2007 OR 2008 using: dt.year=200X to select years 
# use the | to select 2007 or 2008
# part2: groupby([myData["TransDate"].dt.year, "Customer"])
# part 3: take the sum() on the PurchAmount
#.groupby(['Customer'])["PurchAmount"].sum()

In [ ]:
myData["TransDate"].dt.year==2007

In [ ]:
# Question 2. How many customers purchased for more than 50$ in total between 2008 and 2009

# part 1: select rows with year 2008 or 2009 using the loc function

# part 2: groupby("Customer")

# part 3: take the sum() on the PurchAmount

# take the count() on those aggregations that have sum more than 50$, hint [lambda x:x >50]


### 3. Selecting using an aggregating dimension
#### 3.1 Select the first 2 purchases of each customer

In [ ]:
# sort by ascending order of "Customer"
myData.groupby("Customer").head(2).sort_values("Customer")

> Note: use `tail(1)` instead of `head(3)` to select the last purchase of each customer.

#### 3.2 Adding new columns using an aggregating dimension
###### Add a column counting the total quantities purchased by each customer using `pd.transform()`

In [ ]:
# save the total number of occurences per customer in the count column
myData["Count"] = myData.groupby("Customer")["Customer"].transform("size")
myData

#myData.groupby("Customer")["Customer"].transform("count")

###### Add a column counting the cumulative transactions per customer to date

In [ ]:
# counts the cumulative number of transactions per customer
myData["RelDate"] = myData.sort_values('TransDate').groupby("Customer").cumcount() + 1
myData.sort_values(['Customer', 'TransDate'])

#### 3.3 Exercises
###### 1. Add a column to myData with the total number of purchases per customer   
> Hint: Use the `transform()` function.

In [ ]:
# your solution
myData["CountPurch"]= myData.groupby("Customer")["Quantity"].transform("sum")
myData.sample(5)

### Quick intro to Pivot Tables

You can use `pd.pivot_table()` to obtain the same simple aggregation of purchased amount by customer.

In [ ]:
myData.head()

In [ ]:
myData.pivot_table(index='Customer', values='PurchAmount', aggfunc='sum')

But you can also organise your aggregations differently using the `columns` parameter of `pd.pivot_table()`.

In [ ]:
myData.pivot_table(index='Customer', columns=myData.TransDate.dt.year, values='Quantity', aggfunc={'sum'})